# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hawaiian paradise park,19.5933,-154.9731,21.13,84,100,2.57,US,1698507483
1,1,hadibu,12.6500,54.0333,26.73,73,76,4.23,YE,1698507483
2,2,cadale,2.7603,46.3222,28.10,74,100,4.08,SO,1698507484
3,3,la foa,-21.7108,165.8276,21.68,80,78,3.80,NC,1698507484
4,4,waitangi,-43.9535,-176.5597,9.90,83,35,4.92,NZ,1698507484


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[(city_data_df["Max Temp"]>27) & (city_data_df["Max Temp"]<30) & \
                                (city_data_df["Wind Speed"]<5) & (city_data_df["Cloudiness"] ==0)]
# Drop any rows with null values
ideal_city_df.dropna()

# Display sample data
ideal_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,34,saint marys,30.7305,-81.5465,27.79,75,0,4.63,US,1698507213
72,72,ascension,30.2169,-90.9001,27.89,64,0,4.12,US,1698507518
249,249,galliano,29.4422,-90.2992,28.00,54,0,2.57,US,1698507602
359,359,palma soriano,20.2172,-75.9989,29.48,41,0,3.52,CU,1698507654
446,446,veinticinco de mayo,-35.4323,-60.1727,27.21,49,0,3.13,AR,1698507693


### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[["City", "Country","Lng", "Lat", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lng,Lat,Humidity,Hotel Name
34,saint marys,US,-81.5465,30.7305,75,
72,ascension,US,-90.9001,30.2169,64,
249,galliano,US,-90.2992,29.4422,54,
359,palma soriano,CU,-75.9989,20.2172,41,
446,veinticinco de mayo,AR,-60.1727,-35.4323,49,
528,pamandzi,YT,45.2794,-12.7967,78,
551,salalah,OM,54.0924,17.0151,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories" : categories,
    "apiKey": geoapify_key
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
         #If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint marys - nearest hotel: No hotel found
ascension - nearest hotel: Cajun Country Inn
galliano - nearest hotel: Fairfield Inn & Suites by Marriott Cut Off-Galliano
palma soriano - nearest hotel: Hotel Palma
veinticinco de mayo - nearest hotel: Jorly
pamandzi - nearest hotel: Gite Le Madina
salalah - nearest hotel: Muscat International Hotel


,City,Country,Lng,Lat,Humidity,Hotel Name
34,saint marys,US,-81.5465,30.7305,75,No hotel found
72,ascension,US,-90.9001,30.2169,64,Cajun Country Inn
249,galliano,US,-90.2992,29.4422,54,Fairfield Inn & Suites by Marriott Cut Off-Gal...
359,palma soriano,CU,-75.9989,20.2172,41,Hotel Palma
446,veinticinco de mayo,AR,-60.1727,-35.4323,49,Jorly
528,pamandzi,YT,45.2794,-12.7967,78,Gite Le Madina
551,salalah,OM,54.0924,17.0151,74,Muscat International Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [40]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["Hotel Name", "Country"] 
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)